### This notebook reads in radiative flux profile from cloudsat satellite observations to calculate 5-day mean CRH for year 2007 as a example. Detailed methods are described in the subpplementary materials of Liu and Grise 2025.

In [1]:
import os
import numpy as np
from datetime import datetime, timedelta
from pyhdf.SD import SD, SDC
from pyhdf import HDF, VS
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

In [3]:
# Helper function to read Vdata (Latitude, Longitude, Status)
def read_vdata(hdf_file, field_name):
    h = HDF.HDF(hdf_file)
    vs = h.vstart()
    try:
        field_id = vs.attach(vs.find(field_name))
        field_id.setfields(field_name)
        nrecs, _, _, _, _ = field_id.inquire()
        data = np.array(field_id.read(nRec=nrecs)).squeeze()
        field_id.detach()
    except Exception as e:
        print(f"Error reading {field_name}: {e}")
        data = None
    return data

# Function to read datasets from the HDF file (e.g., QR, Height)
def read_hdf_dataset(hdf_file, dataset_name):
    try:
        sd_file = SD(hdf_file, SDC.READ)
        data = sd_file.select(dataset_name)[:]
        sd_file.end()
    except Exception as e:
        print(f"Error reading {dataset_name}: {e}")
        data = None
    return data

# Function to apply quality control
def apply_quality_control(data):
    min_value = data.min()
    data = np.where(data <= min_value, np.nan, data)
    return data

# Function to interpolate and regrid the data
def regrid_data(variable, altitude, alt_grid):
    variable_sorted = np.flip(variable, axis=1)  # Flip along altitude
    altitude_sorted = np.flip(altitude, axis=1)  # Flip along altitude
    regridded_data = np.array([np.interp(alt_grid, altitude_sorted[j, :], variable_sorted[j, :], left=np.nan, right=np.nan) for j in range(len(variable))])
    return regridded_data

# Function to aggregate daily data
def aggregate_daily_data(dates, data_grid, unique_dates):
    numdays = len(unique_dates)
    aggregated_data = np.full((numdays, len(new_lat), len(new_lon), len(alt_grid)), np.nan)
    for k, day in enumerate(unique_dates):
        day_indices = np.where([date == day for date in dates])[0]
        if day_indices.size > 0:
            aggregated_data[k, :, :, :] = np.nanmean(data_grid[day_indices, :, :, :], axis=0)
    return aggregated_data

In [4]:
# Define grid settings
alt_grid = np.arange(0, 15.25, 0.25)
new_lat = np.arange(-88.75, 90, 2.5)
new_lon = np.arange(-178.75, 180, 2.5)

# Set base directory where CloudSat data is located
base_directory = "/bjerknes_raid5/cloudsat/2007"

# List all subdirectories (e.g., 001, 002, etc.)
subdirs = [os.path.join(base_directory, d) for d in sorted(os.listdir(base_directory)) if os.path.isdir(os.path.join(base_directory, d))]

# List all HDF files in the base directory and subdirectories
filelist = []
for subdir in subdirs:
    filelist += [os.path.join(subdir, f) for f in os.listdir(subdir) if f.endswith('.hdf')]

numfiles = len(filelist)
print(f"Number of files: {numfiles}")

# Main processing loop
QR_SWcloud_dia_grid_all = np.full((numfiles, len(new_lat), len(new_lon), len(alt_grid)), np.nan)
QR_LWcloud_dia_grid_all = np.full((numfiles, len(new_lat), len(new_lon), len(alt_grid)), np.nan)
numdate = np.zeros(len(subdirs))  # For 48 days in 2017
dates = []

for i, filepath in enumerate(filelist):
    print(f"Processing file {i+1}/{numfiles}: {filepath}")
    filename = os.path.basename(filepath)
    year = int(filename[:4])
    doy = int(filename[4:7])
    granule = filename.split('_')[1]
    filename_aux = filename.replace("_CS_2B-FLXHR-LIDAR_GRANULE_P2_","_CS_ECMWF-AUX_GRANULE_P1_")
    filepath_aux = '/bjerknes_raid5/cloudsat/aux/' + filename_aux
    date = datetime(year, 1, 1) + timedelta(days=doy - 1)
    dates.append(date)

    # Read geolocation and status fields
    latitude = read_vdata(filepath, 'Latitude')
    longitude = read_vdata(filepath, 'Longitude')
    status = read_vdata(filepath, 'Status')

    # Read Height and QR data from the HDF file
    altitude = read_hdf_dataset(filepath, 'Height') / 1000.0  # Convert to km
    FD = read_hdf_dataset(filepath, 'FD')
    FD = apply_quality_control(FD)
    FU = read_hdf_dataset(filepath, 'FU')
    FU = apply_quality_control(FU)
    FD_NC = read_hdf_dataset(filepath, 'FD_NC')
    FD_NC = apply_quality_control(FD_NC)
    FU_NC = read_hdf_dataset(filepath, 'FU_NC')
    FU_NC = apply_quality_control(FU_NC)

    FN = (FD - FU)/10.0
    FN_NC = (FD_NC - FU_NC)/10.0

    P = read_hdf_dataset(filepath_aux, 'Pressure')
    T = read_hdf_dataset(filepath_aux, 'Temperature')
    P = apply_quality_control(P)
    T = apply_quality_control(T)

    RHO = P/(287.0*T)

    cp = 1004
    dz = 240
    QR_SW_dia = -np.diff(FN[0, :, :])/ (cp*RHO*dz) * 86400  # units of K/day
    QR_LW_dia = -np.diff(FN[1, :, :])/ (cp*RHO*dz) * 86400

    QR_SWcs_dia = -np.diff(FN_NC[0, :, :])/ (cp*RHO*dz) * 86400  # units of K/day
    QR_LWcs_dia = -np.diff(FN_NC[1, :, :])/ (cp*RHO*dz) * 86400

    QR_SWcloud_dia = QR_SW_dia - QR_SWcs_dia
    QR_LWcloud_dia = QR_LW_dia - QR_LWcs_dia

    # Regrid QRS and QRL to standard altitude grid
    QR_SWcloud_dia_regrid = regrid_data(QR_SWcloud_dia, altitude, alt_grid)
    QR_LWcloud_dia_regrid = regrid_data(QR_LWcloud_dia, altitude, alt_grid)

    # Grid the data into latitude and longitude bins
    QR_SWcloud_dia_grid = np.full((len(new_lat), len(new_lon), len(alt_grid)), np.nan)
    QR_LWcloud_dia_grid = np.full((len(new_lat), len(new_lon), len(alt_grid)), np.nan)

    for m in range(len(new_lat) - 1):
        for n in range(len(new_lon) - 1):
            indices = np.where(
                (latitude >= new_lat[m]) & (latitude < new_lat[m + 1]) &
                (longitude >= new_lon[n]) & (longitude < new_lon[n + 1])
            )[0]
            if indices.size > 0:
                QR_SWcloud_dia_grid[m + 1, n + 1, :] = np.nanmean(QR_SWcloud_dia_regrid[indices, :], axis=0)
                QR_LWcloud_dia_grid[m + 1, n + 1, :] = np.nanmean(QR_LWcloud_dia_regrid[indices, :], axis=0)

    # Store the gridded data
    QR_SWcloud_dia_grid_all[i, :, :, :] = QR_SWcloud_dia_grid
    QR_LWcloud_dia_grid_all[i, :, :, :] = QR_LWcloud_dia_grid


# Compute daily zonal mean (for QRL and QRS)
unique_dates = np.unique(dates)
QR_SWcloud_dia_grid_daily = aggregate_daily_data(dates, QR_SWcloud_dia_grid_all, unique_dates)
QR_LWcloud_dia_grid_daily = aggregate_daily_data(dates, QR_LWcloud_dia_grid_all, unique_dates)

QR_SWcloud_dia_grid_daily_zonalmean = np.nanmean(QR_SWcloud_dia_grid_daily, axis=2)
QR_LWcloud_dia_grid_daily_zonalmean = np.nanmean(QR_LWcloud_dia_grid_daily, axis=2)

# Save data to a numpy .npz file
np.savez('QR_cloud_2007.npz',
         QR_SWcloud_dia_grid_daily=QR_SWcloud_dia_grid_daily,
         QR_LWcloud_dia_grid_daily=QR_LWcloud_dia_grid_daily,
         QR_SWcloud_dia_grid_daily_zonalmean=QR_SWcloud_dia_grid_daily_zonalmean,
         QR_LWcloud_dia_grid_daily_zonalmean=QR_LWcloud_dia_grid_daily_zonalmean,
         alt_grid=alt_grid,
         new_lat=new_lat,
         new_lon=new_lon,
         numdate=unique_dates)

print("Processing complete. Data saved to QR_cloud_2007.npz")

Number of files: 4702
Processing file 1/4702: /bjerknes_raid5/cloudsat/2007/001/2007001040927_03609_CS_2B-FLXHR-LIDAR_GRANULE_P2_R05_E02_F00.hdf


/tmp/ipykernel_53193/3031461719.py:89: RuntimeWarning: Mean of empty slice
  QR_SWcloud_dia_grid[m + 1, n + 1, :] = np.nanmean(QR_SWcloud_dia_regrid[indices, :], axis=0)
/tmp/ipykernel_53193/3031461719.py:90: RuntimeWarning: Mean of empty slice
  QR_LWcloud_dia_grid[m + 1, n + 1, :] = np.nanmean(QR_LWcloud_dia_regrid[indices, :], axis=0)


Processing file 2/4702: /bjerknes_raid5/cloudsat/2007/001/2007001122353_03614_CS_2B-FLXHR-LIDAR_GRANULE_P2_R05_E02_F00.hdf
Processing file 3/4702: /bjerknes_raid5/cloudsat/2007/001/2007001203818_03619_CS_2B-FLXHR-LIDAR_GRANULE_P2_R05_E02_F00.hdf
Processing file 4/4702: /bjerknes_raid5/cloudsat/2007/001/2007001072713_03611_CS_2B-FLXHR-LIDAR_GRANULE_P2_R05_E02_F00.hdf
Processing file 5/4702: /bjerknes_raid5/cloudsat/2007/001/2007001172031_03617_CS_2B-FLXHR-LIDAR_GRANULE_P2_R05_E02_F00.hdf
Processing file 6/4702: /bjerknes_raid5/cloudsat/2007/001/2007001090607_03612_CS_2B-FLXHR-LIDAR_GRANULE_P2_R05_E02_F00.hdf
Processing file 7/4702: /bjerknes_raid5/cloudsat/2007/001/2007001221711_03620_CS_2B-FLXHR-LIDAR_GRANULE_P2_R05_E02_F00.hdf
Processing file 8/4702: /bjerknes_raid5/cloudsat/2007/001/2007001235604_03621_CS_2B-FLXHR-LIDAR_GRANULE_P2_R05_E02_F00.hdf
Processing file 9/4702: /bjerknes_raid5/cloudsat/2007/001/2007001023034_03608_CS_2B-FLXHR-LIDAR_GRANULE_P2_R05_E02_F00.hdf
Processing file 

/tmp/ipykernel_53193/4128240586.py:47: RuntimeWarning: Mean of empty slice
  aggregated_data[k, :, :, :] = np.nanmean(data_grid[day_indices, :, :, :], axis=0)
/tmp/ipykernel_53193/3031461719.py:102: RuntimeWarning: Mean of empty slice
  QR_SWcloud_dia_grid_daily_zonalmean = np.nanmean(QR_SWcloud_dia_grid_daily, axis=2)
/tmp/ipykernel_53193/3031461719.py:103: RuntimeWarning: Mean of empty slice
  QR_LWcloud_dia_grid_daily_zonalmean = np.nanmean(QR_LWcloud_dia_grid_daily, axis=2)


Processing complete. Data saved to QR_cloud_2007.npz


### load QR_cloud_2007.npz and convert it to a netcdf file

In [6]:
CRH_binned = np.load('QR_cloud_2007.npz', allow_pickle=True)
CRHlw_grid_daily_zonalmean = CRH_binned['QR_LWcloud_dia_grid_daily_zonalmean']
CRHsw_grid_daily_zonalmean = CRH_binned['QR_SWcloud_dia_grid_daily_zonalmean']
CRH_grid_daily_zonalmean = CRHlw_grid_daily_zonalmean + CRHsw_grid_daily_zonalmean

In [7]:
# Define coordinates (you may need to adapt these based on your data)
alt_grid = CRH_binned['alt_grid']
new_lat = CRH_binned['new_lat']
time = CRH_binned['numdate']  # Assuming numdate holds time information

# Create an xarray DataArray
CRHlw_grid_da = xr.DataArray(
    CRHlw_grid_daily_zonalmean,
    coords={'time': time, 'height': alt_grid, 'lat': new_lat},
    dims=['time', 'lat', 'height'],
    name='CRH_LWcloud'
)

CRHlw_grid_da.to_netcdf('/bjerknes_raid5/cloudsat/CRH_LW_CloudSat_2007.nc')

In [4]:
CRH_binned = np.load('QR_cloud_2007.npz', allow_pickle=True)
CRHlw_grid_daily_zonalmean = CRH_binned['QR_LWcloud_dia_grid_daily_zonalmean']
CRHsw_grid_daily_zonalmean = CRH_binned['QR_SWcloud_dia_grid_daily_zonalmean']
CRH_grid_daily_zonalmean = CRHlw_grid_daily_zonalmean + CRHsw_grid_daily_zonalmean

alt_grid = CRH_binned['alt_grid']
new_lat = CRH_binned['new_lat']
time = CRH_binned['numdate']  # Assuming numdate holds time information

# Create an xarray DataArray
CRHlw_grid_da = xr.DataArray(
    CRHlw_grid_daily_zonalmean,
    coords={'time': time, 'height': alt_grid, 'lat': new_lat},
    dims=['time', 'lat', 'height'],
    name='CRH_LWcloud'
)

CRHlw_grid_da.to_netcdf('/bjerknes_raid5/cloudsat/CRH_LW_CloudSat_2007.nc')

other years can be processed in a simlar way...

#### combine years together

In [ ]:
file_2007 = xr.open_dataset("/bjerknes_raid5/cloudsat/CRH_LW_CloudSat_2007.nc")
file_2008 = xr.open_dataset("/bjerknes_raid5/cloudsat/CRH_LW_CloudSat_2008.nc")
file_2009 = xr.open_dataset("/bjerknes_raid5/cloudsat/CRH_LW_CloudSat_2009.nc")
file_2010 = xr.open_dataset("/bjerknes_raid5/cloudsat/CRH_LW_CloudSat_2010.nc")

# Concatenate the datasets along the time dimension
combined = xr.concat([file_2007, file_2008, file_2009, file_2010], dim="time")

# Save the combined dataset to a new NetCDF file
combined.to_netcdf("../data/CRH_LW_CloudSat_combined.nc")